In [1]:
import sys
import logging 
import pandas as pd
import numpy as np
import xgboost as xgb
from utils import read_convert_file 
from CONSTANTS import FILE_PATH, xgb_params
from sklearn.pipeline import Pipeline
import joblib

In [2]:
# Load the trained pipeline
loaded_pipeline = joblib.load('ml_pipeline.pkl')    

In [24]:
# New sample data for prediction
new_data = pd.DataFrame({'BusinessDate': ['2025-03-25'], 'FiscalYearNumber': [2025], 'FiscalPeriodInYearNumber': [5], 'DayPartCode' : ['Afternoon: 2 PM - 5 PM'], 
                         'StoreNumber': [10004], 'CustomerOrderChannelName' :['OTW'], 
                        'RunKey' : [10003] })


In [ ]:
processed_new_data = new_data.copy() #read the file.
processed_new_data['BusinessDate'] = pd.to_datetime(processed_new_data['BusinessDate'])
processed_new_data = processed_new_data.set_index('BusinessDate')
# processed_new_data = loaded_pipeline.named_steps['read_convert_file'].transform(processed_new_data)
processed_new_data = loaded_pipeline.named_steps['time_features'].transform(processed_new_data)
processed_new_data = loaded_pipeline.named_steps['convert_dtype'].transform(processed_new_data)
processed_new_data = loaded_pipeline.named_steps['drop_columns'].transform(processed_new_data)




In [26]:
# Make predictions using the loaded pipeline
predictions = loaded_pipeline.named_steps['xgb'].predict(processed_new_data)

In [27]:
print("Predictions:", predictions)

Predictions: [378.18878]


In [9]:
# Predict using the loaded pipeline
xgb_model  = loaded_model.predict(new_data)  # Apply all transformations except model training

# Convert to DMatrix
# dnew = xgb.DMatrix(processed_new_data)


# prediction = loaded_model.predict(new_data)
# print("Prediction for new data:", prediction)

AttributeError: This 'Pipeline' has no attribute 'predict'

In [13]:
new_data

,BusinessDate,FiscalYearNumber,FiscalPeriodInYearNumber,DayPartCode,StoreNumber,CustomerOrderChannelName,RunKey
0,2025-13-03,2025,5,Mid-Morning: 9 AM - 11 AM,10003,OTW,10003


In [ ]:
class XGBTrainer:
    """ Wrapper class for training an XGBoost model in a scikit-learn pipeline.  """
    
    def __init__ (self, xgb_params, num_boost_round =  999):
        self.xgb_params = xgb_params 
        self.num_boost_round = num_boost_round
        self.model = None 
    
    def fit (self, X_y, y= None):
        
        """
        Trains an XGBoost model.

        Args:
            X_y (tuple): Tuple containing (X_train, y_train, X_test, y_test)

        Returns:
            self: Trained model
        """
        
        X_train, y_train, X_test, y_test = X_y  
        dtrain = xgb.DMatrix(X_train, label=y_train)
        dtest = xgb.DMatrix(X_test, label=y_test) 
        self.model =  xgb.train(
                self.xgb_params,
                dtrain,
                num_boost_round=self.num_boost_round,
                evals=[(dtest, "Test")],
                early_stopping_rounds=10) 
        return self 
    def transform(self, X_y):
        """ Returns the trained XGBoost model. """
        return self.model 
# -------------------------------------------------------------------------------